In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from datetime import datetime as dt, timedelta
from pandas_profiling import ProfileReport
plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns; sns.set()  # for plot styling
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go
import datetime
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
display(HTML("<style>.container { width:100% !important; }</style>"))
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error,r2_score
import math
from sklearn.model_selection import train_test_split
import xlsxwriter
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from openpyxl import load_workbook
# writer = pd.ExcelWriter('output.xlsx', engine = 'xlsxwriter')
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
SCOPE = ['https://www.googleapis.com/auth/drive']
CREDENTIALS = ServiceAccountCredentials.from_json_keyfile_name('../expanded-bonbon-239019-ed390949c71b.json', SCOPE)

In [57]:
xls = pd.ExcelFile('Apartments2021.xlsx')
century = pd.read_excel(xls, 'century21')
elite = pd.read_excel(xls, 'elite')
futurehome = pd.read_excel(xls, 'futurehome')
mei = pd.read_excel(xls, 'mei')
zv_zone = pd.read_excel(xls, 'zevendesimi i zonave')

In [77]:
elite.loc[elite.century_zone == "Not mapped"].zone

23                    Rr. Xhanfize Keko
24                    Rr. Xhanfize Keko
54                          Thanas Ziko
55                          Thanas Ziko
57                          Thanas Ziko
                     ...               
1377                   Rr. Todi Shkurti
1380                    Rr.Irfan Tomini
1381                  Rr. Tre Deshmoret
1385    Prane Gjykates se Faktit Tirane
1389                  Rr. Mihal Grameno
Name: zone, Length: 242, dtype: object

In [78]:
oc = pd.DataFrame()
for zone in zv_zone.loc[zv_zone["Zona në sistem"] != ""]["Zona në sistem"]:
    
#     Ratios = process.extract(r, path_to_map['Column'].dropna().unique()) # Extracts all matches for the given string
    highest = process.extractOne(zone, elite.loc[elite.century_zone == "Not mapped"].zone.drop_duplicates() ) # Extracts the highest match for the given string
    row = {
        'elite': zone,
        'century': highest[0],
        'Percentage': highest[1]
    }
    oc = oc.append(row, ignore_index=True)
    
elite_zones = oc.rename(columns={
    'String': 'century'
})[['century', 'elite', 'Percentage']]

In [89]:
zv_zone.loc[zv_zone["Zona në sistem"].isin(elite_zones.loc[(elite_zones.Percentage==90)]["elite"])]
# elite_zones.loc[(elite_zones.Percentage>90)][["elite",'century', "Percentage"]]

,Zona në sistem,Zëvëndësimi I saktë
14,Nobis,Liqeni i Tiranës
26,Rr. Industriale,Autostrada Tiranë-Durrës
104,Uzina traktori,Shkozë
106,Zone industriale,Autostrada Tiranë-Durrës
112,Pediatria,Spitali QSUT (Nënë Tereza)
114,tregu elektrik,Tregu Elektrik
178,Liqeni,Liqeni i Tiranës
183,Petronini,Gjimnazi Petro Nini Luarasi
184,Blv.Bajram Curri,Bulevardi
185,Petrele,Petrelë


In [90]:
elite_zones.loc[(elite_zones.Percentage==90)][["elite",'century', "Percentage"]]

,elite,century,Percentage
14,Nobis,Pranë Nobis,90.0
26,Rr. Industriale,tr,90.0
104,Uzina traktori,tr,90.0
106,Zone industriale,tr,90.0
112,Pediatria,tr,90.0
114,tregu elektrik,tr,90.0
178,Liqeni,Liqeni Artificial i Tiranës,90.0
183,Petronini,tr,90.0
184,Blv.Bajram Curri,"Prane Piramides, Blv Bajram Curri",90.0
185,Petrele,tr,90.0


In [42]:
oc = pd.DataFrame()
for zone in century_futurehome.loc[century_futurehome.futurehome != ""].futurehome:
    
#     Ratios = process.extract(r, path_to_map['Column'].dropna().unique()) # Extracts all matches for the given string
    highest = process.extractOne(zone, century_futurehome.century ) # Extracts the highest match for the given string
    row = {
        '': zone,
        'century': highest[0],
        'Percentage': highest[1]
    }
    oc = oc.append(row, ignore_index=True)
    
futurehome_zones = oc.rename(columns={
    'String': 'century'
})[['century', 'futurehome', 'Percentage']]

In [54]:
# futurehome_zones.loc[(futurehome_zones.Percentage >= 60) & (futurehome_zones.Percentage < 70) ][['elite',  'century',"Percentage"]]
futurehome_zones.loc[(futurehome_zones.Percentage>70) & (futurehome_zones.Percentage<80)][["futurehome",'century', "Percentage"]]

,futurehome,century,Percentage
2,Rruga Zenel Baboçi,Rruga e Elbasanit,71.0
8,Shkolla Partizani,Shkozë,72.0
13,Garden Residence Turdiu,Garda,72.0
49,Rruga Mihal Duri,Rruga e Durrësit,71.0


In [10]:
elite_zones = elite_zones.sort_values("Percentage", ascending=False)

elite.loc[elite.zone.isin(elite_zones.loc[elite_zones.Percentage == 100]["elite"])]["century_zone"] = elite_zones.loc[elite_zones.Percentage == 100]["century"]
elite.loc[elite.zone.isin(elite_zones.loc[elite_zones.Percentage == 100]["elite"])]
elite_zones.loc[(elite_zones.Percentage == 100) & elite_zones.elite.isin()]["century"]

,propertiesid,property_type,property_status,availability,country,division,city,current_zones,zone,century_zone,price,interior_area,gros_area,bedrooms,bathrooms,other_rooms,year_of_construction,year_of_renovation
0,15085,Apartment,Used,Available,Albania,Tirana,Tirana,Rruga e Kavajes,Rruga e Kavajes,NaN,95000,90,95,2,1,0,0,0.0
1,15084,Apartment,Used,Available,Albania,Tirana,Tirana,Komuna e Parisit,Komuna e Parisit,NaN,179800,124,0,2,2,0,0,0.0
3,15003,Apartment,Used,Available,Albania,Tirana,Tirana,Rruga e Elbasanit,Rruga e Elbasanit,NaN,125000,95,0,2,1,0,0,0.0
6,14993,Apartment,New,Available,Albania,Tirana,Tirana,Rruga e Kavajes,Rruga e Kavajes,NaN,63000,73,73,1,1,0,0,0.0
9,14967,Apartment,Used,Available,Albania,Tirana,Tirana,Brryli,Brryli,NaN,62000,65,65,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,6662,Apartment,New,Sold,Albania,Tirana,Tirana,Kodra e Diellit,Kodra e Diellit,NaN,120000,120,0,2,2,0,0,0.0
1313,6648,Apartment,New,Sold,Albania,Tirana,Tirana,Rruga e Kavajes,Rruga e Kavajes,NaN,197500,158,0,2,2,0,0,0.0
1332,6544,Apartment,New,Sold,Albania,Tirana,Tirana,Liqeni i Thatë,Liqeni i Thatë,NaN,55000,103,0,2,0,0,0,0.0
1353,6456,Apartment,New,Rented,Albania,Tirana,Tirana,Komuna e Parisit,Komuna e Parisit,NaN,85000,111,0,2,2,0,0,0.0
